## Step 1 : Import librairies

In [ ]:
import pandas as pd
import cv2
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import keyboard
import time

## Step 2 : Retrieve data

In [ ]:
TABLENAME = "Test_Crash_Table.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(TABLENAME)
df

In [ ]:
# Select only frame columns
frame_cols = [f"frame_{i}" for i in range(1, 51)]

# Select only the frame columns from the DataFrame
frame_data = df[frame_cols]

# Flatten data into a single list
labels = frame_data.values.flatten()

print(labels)
print(labels.shape)

In [ ]:
IMAGE_PATH = "C:\\Users\\sacha\\OneDrive\\Bureau\\CrashBest"
BATCH_SIZE = int(len(labels))

image_paths = list(Path(IMAGE_PATH).glob("*.jpg"))[:BATCH_SIZE]
print(image_paths[0])
print(len(image_paths))

In [ ]:
img_width = 256
img_height = 256

# create a dataset class for the crash frames
class CrashFrameDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.cvtColor(cv2.imread(str(image_path)), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (img_width, img_height))
        image = Image.fromarray(image.astype('uint8'))
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([
    transforms.ToTensor()
])

# Use the existing transform variable defined in a previous cell
dataset = CrashFrameDataset(image_paths, labels, transform=transform)
len(dataset)

# Step 3 : Build a Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CrashDetection(nn.Module):
    def __init__(self, input_channels=3, filter_base=8, input_height=256, input_width=256):
        super(CrashDetection, self).__init__()
        self.input_channels = input_channels
        self.filter_base = filter_base
        self.input_height = input_height
        self.input_width = input_width

        # CNN layers
        self.conv1_1 = nn.Conv2d(input_channels, filter_base, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(filter_base, filter_base, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=4)

        self.conv2_1 = nn.Conv2d(filter_base, filter_base * 2, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(filter_base * 2, filter_base * 2, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=4)

        self.conv3_1 = nn.Conv2d(filter_base * 2, filter_base * 4, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(filter_base * 4, filter_base * 4, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        # Placeholder for output size
        self._to_linear = None
        self._compute_flattened_size()

        self.fc1 = nn.Linear(self._to_linear, 128)
        self.fc2 = nn.Linear(128, 16)
        self.fc3 = nn.Linear(16, 1)

    def _compute_flattened_size(self):
        with torch.no_grad():
            x = torch.zeros(1, self.input_channels, self.input_height, self.input_width)
            x = F.relu(self.conv1_1(x))
            x = F.relu(self.conv1_2(x))
            x = self.pool1(x)

            x = F.relu(self.conv2_1(x))
            x = F.relu(self.conv2_2(x))
            x = self.pool2(x)

            x = F.relu(self.conv3_1(x))
            x = F.relu(self.conv3_2(x))
            x = self.pool3(x)

            self._to_linear = x.view(1, -1).shape[1]

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool1(x)
        #print('Output shape of layer 1', x.shape)

        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool2(x)
        #print('Output shape of layer 2', x.shape)

        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = self.pool3(x)
        #print('Output shape of layer 3', x.shape)

        x = x.reshape(x.size(0), -1)
        #print('Shape required to pass to Linear Layer', x.shape)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = CrashDetection().to(device)

# Step 4 : Load and try the model with new data

In [ ]:
import glob

model_paths = glob.glob("Models/CarCrashPytorch_*.keras")
model_path = model_paths[len(model_paths) - 1]
print(f"Loading model from: {model_path}")

model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode

In [ ]:
import matplotlib.pyplot as plt
import cv2
import torch
from IPython.display import clear_output, display

# Set up the figure for displaying images
fig, ax = plt.subplots()
current_index = 0
count_pred_crash = 0
count_pred_noCrash = 0
count_crash = 0
count_noCrash = 0

def update_image():
    global current_index
    global count_pred_crash, count_pred_noCrash, count_crash, count_noCrash
    image, label = dataset[current_index]
    
    # Convert tensor to numpy and transpose to (H, W, C) for imshow
    if isinstance(image, torch.Tensor):
        image = image.cpu().numpy().transpose(1, 2, 0)
    else:
        image = np.array(image)
        
    ax.clear()
    ax.imshow(image)
    ax.set_title(f"Label: {label}")
    ax.axis("off")

    clear_output(wait=True)
    display(fig)

    # Convert image to the format expected by the model
    image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2).to(device) / 255.0

    model.eval()
    with torch.no_grad():
        y_sigmoid = model(image_tensor)
        y_pred = torch.argmax(y_sigmoid, axis=-1).cpu().numpy()

    if y_pred == 0:
        print(f"No crash detected")
        count_pred_noCrash += 1
    else:
        print(f"Crash detected")
        count_pred_crash += 1
    
    if label == 0:
        count_noCrash += 1
    else:
        count_crash += 1

# loop through the dataset and display images
for i in range(len(image_paths)):
    current_index = i
    update_image()
    print(f"Current index: {current_index + 1}/{len(image_paths)}")
    print(f"Total no-crash detected: {count_pred_noCrash}")
    print(f"Total crashes detected: {count_pred_crash}")
    print(f"Total real no-crash: {count_noCrash}")
    print(f"Total real crash: {count_crash}")

    if keyboard.is_pressed("esc"): # Exit the loop (Escape Key)
        clear_output(wait=True)
        print(f"Current index: {current_index + 1}/{len(image_paths)}")
        print(f"Total no-crash detected: {count_pred_noCrash}")
        print(f"Total crashes detected: {count_pred_crash}")
        print(f"Total real no-crash: {count_noCrash}")
        print(f"Total real crash: {count_crash}")
        
        break
    
    # time.sleep(0.05)